In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME DONE #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# so just keeping creds here for the CRUD module hook-in
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter()

# so we grab everything first; filters will narrow this down later
df = pd.DataFrame.from_records(db.read({}))

# dash doesn’t love ObjectID in _id, so we drop it to avoid type issues
df.drop(columns=['_id'], inplace=True)

## Debug lines if needed
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# so this pulls in the logo and encodes it for inline use
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# quick ID tag so the client knows it’s mine/versioned
UNIQUE_ID = "3232" 
YOUR_NAME = "Allen"  

# layout: title, logo, credits, filters, table, then chart/map side-by-side
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),

    # so this keeps the brand mark stays small and clean up top
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '70px'}),

    # dev signature line
    html.Div(f"Developed by: {YOUR_NAME} — Unique ID: {UNIQUE_ID}", style={'fontWeight': 'bold', 'margin': '6px 0'}),

    html.Hr(),

    # so: one-click rescue-type filters for the client
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            inline=True,
            style={'padding': '6px 0'}
        )
    ),

    html.Hr(),

    # need this so the table is interactive (sort/filter/select) right out of the box
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'minWidth': '120px', 'whiteSpace': 'normal'}
    ),

    html.Br(),
    html.Hr(),

    # Use this so thechart on the left, map on the right — simple two-column flex
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################

# so design-wise: these filters line up with the course mission profiles the client asked for
# (water, mountain, disaster) — helps the user click once and see the right rescue-ready dogs

# had to centralized query builder so filters stay readable and easy to tweak 
def build_query(filter_type):
    if filter_type == 'water':
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                              "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                              "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        return {}  # reset shows all data


# so when the radio changes, we hit Mongo again with the right filter
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = build_query(filter_type)
    dff = pd.DataFrame.from_records(db.read(query))

    # if nothing matches, just hand the table an empty list (keeps it calm)
    if dff.empty:
        return []

    # safety: drop _id if it came back
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    return dff.to_dict('records')


#  this chart follows whatever is showing in the table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or 'outcome_type' not in dff.columns:
        return []

    # quick read on outcomes; pie keeps it super visual for the client
    fig = px.pie(dff, names='outcome_type', title='Outcome Types (Current Filter)')
    return [dcc.Graph(figure=fig)]


# have this so the table gives a little visual cue when a column’s selected
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# so the map updates to the currently selected row (marker + popup)
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)

    # single selection, so just grab the first index
    if index is None:
        row = 0
    else:
        row = index[0]

    # Austin, TX center — coordinates from the row drive the marker
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # breed
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])     # name
                ])
            ])
        ])
    ]


# had to use 8051 to avoids the default port clash
app.run_server(mode='inline', port=8051, debug=False)


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2025 00:33:16] "GET /_alive_1c375ea4-7a19-4d39-bd7c-8927b1ef8c28 HTTP/1.1" 200 -


127.0.0.1 - - [19/Oct/2025 00:33:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:17] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2025 00:33:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:18] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2025 00:33:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 00:33:23] "POST /_dash-update-compo